Условие: создайте csv файл с таким содержимым:

title,author,genre,sales,year

"1984", "George Orwell", "Science Fiction", 5000, 1949

"The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954

"To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960

"The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951

"The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925

**Задание:***

**— Используя Spark прочитайте данные из файла csv.**

**— Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.**

**— Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.**

**— Отсортируйте данные по общему объему продаж в порядке убывания.**

**— Выведите результаты на экран.**

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=c7026fd263fb0465505807f8abf9c2f7b6ccae331c7ae3ef18c4886695eb09e2
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import sum, avg, when, max, day, col, mean
import pyspark.sql.functions as fn
from pyspark.sql.types import DoubleType

In [6]:
# Создание Spark сессии
spark = SparkSession.builder.appName("Books_statistics").getOrCreate()

In [37]:
# Загрузка данных
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("Seminar5_books.csv")
print("Dataset:")
df.show()

Dataset:
+--------------------+--------------------+------------------+------+------+
|               title|              author|             genre| sales|  year|
+--------------------+--------------------+------------------+------+------+
|                1984|     "George Orwell"| "Science Fiction"|5000.0|1949.0|
|The Lord of the R...|    "J.R.R. Tolkien"|         "Fantasy"|3000.0|1954.0|
|To Kill a Mocking...|        "Harper Lee"| "Southern Gothic"|4000.0|1960.0|
|The Catcher in th...|     "J.D. Salinger"|           "Novel"|2000.0|1951.0|
|    The Great Gatsby| "F. Scott Fitzge...|           "Novel"|4500.0|1925.0|
+--------------------+--------------------+------------------+------+------+



In [38]:
# Фильтрация книг, продажи по которым превышают 3000 экземпляров
filtered_df = df.filter(df.sales > 3000)
print("Фильтрация по продажам (более 3000 экземпляров):")
filtered_df.show()

Фильтрация по продажам (более 3000 экземпляров):
+--------------------+--------------------+------------------+------+------+
|               title|              author|             genre| sales|  year|
+--------------------+--------------------+------------------+------+------+
|                1984|     "George Orwell"| "Science Fiction"|5000.0|1949.0|
|To Kill a Mocking...|        "Harper Lee"| "Southern Gothic"|4000.0|1960.0|
|    The Great Gatsby| "F. Scott Fitzge...|           "Novel"|4500.0|1925.0|
+--------------------+--------------------+------------------+------+------+



In [39]:
# Группировка данных по жанру и вычисление общего объема продаж для каждого жанра
window_spec = Window.partitionBy("genre")
sum_sales_by_genre_df = df.withColumn("sum_sales_by_genre", sum(col("sales")).over(window_spec)).show()

+--------------------+--------------------+------------------+------+------+------------------+
|               title|              author|             genre| sales|  year|sum_sales_by_genre|
+--------------------+--------------------+------------------+------+------+------------------+
|The Lord of the R...|    "J.R.R. Tolkien"|         "Fantasy"|3000.0|1954.0|            3000.0|
|The Catcher in th...|     "J.D. Salinger"|           "Novel"|2000.0|1951.0|            6500.0|
|    The Great Gatsby| "F. Scott Fitzge...|           "Novel"|4500.0|1925.0|            6500.0|
|                1984|     "George Orwell"| "Science Fiction"|5000.0|1949.0|            5000.0|
|To Kill a Mocking...|        "Harper Lee"| "Southern Gothic"|4000.0|1960.0|            4000.0|
+--------------------+--------------------+------------------+------+------+------------------+



In [40]:
# Сортировка данных по общему объему продаж в порядке убывания и вывод результатов:
print("Общие объемы продаж по жанрам в порядке убывания:")
window_spec = Window.partitionBy("genre")
sum_sales_by_genre_df = df.withColumn("sum_sales_by_genre", sum(col("sales")).over(window_spec))
result = sum_sales_by_genre_df.select(["genre", "sum_sales_by_genre"]).distinct() \
    .orderBy(sum_sales_by_genre_df.sum_sales_by_genre.desc()).show()

Общие объемы продаж по жанрам в порядке убывания:
+------------------+------------------+
|             genre|sum_sales_by_genre|
+------------------+------------------+
|           "Novel"|            6500.0|
| "Science Fiction"|            5000.0|
| "Southern Gothic"|            4000.0|
|         "Fantasy"|            3000.0|
+------------------+------------------+

